<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/jupyter_best_practices_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pylab as plt

In [8]:
dataset_url='https://raw.githubusercontent.com/martin-fabbri/colab-notebooks/master/data/wowah_data.csv'
df = pd.read_csv(dataset_url)
df.head()

,char,level,race,charclass,zone,guild,timestamp
0,59425,1,Orc,Rogue,Orgrimmar,165,01/01/08 00:02:04
1,65494,9,Orc,Hunter,Durotar,-1,01/01/08 00:02:04
2,65325,14,Orc,Warrior,Ghostlands,-1,01/01/08 00:02:04
3,65490,18,Orc,Hunter,Ghostlands,-1,01/01/08 00:02:04
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,01/01/08 00:02:09


In [15]:
def loggg(f):
  def wrapper(df, *args, **kwargs):
    tic = dt.datetime.now()
    result = f(df, *args, **kwargs)
    toc = dt.datetime.now()
    n_char = result.groupby('char').agg(n_char=('char', np.size)).shape
    print(f'{f.__name__} took: {toc - tic} shape={result.shape} n_char={n_char[0]}')
    return result
  return wrapper

@loggg
def start_pipeline(df):
  return df.copy()

@loggg
def clean_dataset(df):
  df.columns = [col_name.lstrip() for col_name in df.columns ]
  return df

@loggg
def remove_outliers(df):
  return df

clean_df = (df
 .pipe(start_pipeline)
 .pipe(clean_dataset)
 .pipe(remove_outliers))

start_pipeline took: 0:00:00.037775 shape=(1606208, 7) n_char=9466
clean_dataset took: 0:00:00.000184 shape=(1606208, 7) n_char=9466
remove_outliers took: 0:00:00.000012 shape=(1606208, 7) n_char=9466


In [4]:
df.columns


Index(['char', ' level', ' race', ' charclass', ' zone', ' guild',
       ' timestamp'],
      dtype='object')

In [3]:
df.columns = [col_name.lstrip() for col_name in df.columns ]
df.columns

Index(['char', 'level', 'race', 'charclass', 'zone', 'guild', 'timestamp'], dtype='object')

In [4]:
df2 = (df
 .groupby('char')
 .agg(n_row=('level', np.size),
      min_level=('level', np.min),
      max_level=('level', np.max),
      guild=('guild', np.max)))
df2.head()

,n_row,min_level,max_level,guild
char,,,,
7,231,54,60,282
9,943,70,70,79
19,895,69,70,65
21,217,70,70,205
22,17,62,62,5


In [5]:
df3 = (df.groupby('guild').agg(n_char=('char', 'nunique')))
df3.head()

,n_char
guild,
-1,6931
1,1
3,21
4,18
5,229


In [6]:
df4 = df2.merge(df3.reset_index())
df.head()

,char,level,race,charclass,zone,guild,timestamp
0,59425,1,Orc,Rogue,Orgrimmar,165,01/01/08 00:02:04
1,65494,9,Orc,Hunter,Durotar,-1,01/01/08 00:02:04
2,65325,14,Orc,Warrior,Ghostlands,-1,01/01/08 00:02:04
3,65490,18,Orc,Hunter,Ghostlands,-1,01/01/08 00:02:04
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,01/01/08 00:02:09


In [7]:
plt.scatter(x=df4['n_row'], y=df4['level_inc']);

KeyError: ignored

In [0]:
df4['level_inc'] = df4['max_level'] - df4['min_level']